# **資産収益率の計算**

## **1. Googleドライブをマウント**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **2. Googleドライブ内のCSVファイルを読み込み**

In [ ]:
import pandas as pd
import numpy as np

# Googleドライブのパスを指定して読み込み
file_path = '/content/drive/My Drive/csv/input/stock_price.csv'

df = pd.read_csv(file_path)

# データフレームの内容を表示
print(df.head())

## **3. 証券コードごとに収益率を計算し、平均値、標準偏差、最小値、最大値を計算**


In [ ]:
# Date列を日付型に変換
df['Date'] = pd.to_datetime(df['Date'])

# 証券Codeごとにグループ化して収益率を計算
df['Simple_Return'] = df.groupby('Code')['Price'].pct_change()

print(df.head())

stats = df.groupby('Code')['Simple_Return'].agg(['mean', 'std', 'min', 'max'])

print(stats)

## **4. 累積リターンのグラフを作成**

In [ ]:
import matplotlib.pyplot as plt

# 累積リターンを計算
df['Cumulative_Return'] = (1 + df['Simple_Return']).groupby(df['Code']).cumprod()

# Codeごとにグラフを作成
plt.figure(figsize=(10, 6))

for code in df['Code'].unique():
    plt.plot(df[df['Code'] == code]['Date'], df[df['Code'] == code]['Cumulative_Return'], label=code)

# グラフのタイトルとラベル
plt.title('Cumulative Return Over Time')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()

# グラフを表示
plt.show()

# **収益率のリスク表現**

## **5. リターンデータからピボットテーブルを作成し、リターンの平均値、共分散行列を計算**

In [ ]:
# ピボットテーブルを作成
pivot_table = df.pivot(index='Date', columns='Code', values='Simple_Return')

# ピボットテーブルを表示
print(pivot_table.head())

# リターンの平均値を計算
expected_returns = pivot_table.mean()

# 共分散行列を計算
cov_matrix = pivot_table.cov()

# リターンの平均値を表示
print(expected_returns)

# 共分散行列を表示
print(cov_matrix)

## **6. 特定のウェイトを設定し、ポートフォリオ全体の平均リターン、分散を計算**

In [ ]:
import numpy as np

# 各Codeのウェイトを定義
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# ポートフォリオの平均収益率を計算
portfolio_return = np.dot(weights, expected_returns)

# ポートフォリオの分散を計算
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))

# ポートフォリオの平均収益率と分散を表示
print("Portfolio Return: ", portfolio_return)
print("Portfolio Variance: ", portfolio_variance)
print("Portfolio Standard Deviation: ", np.sqrt(portfolio_variance))

#**平均・分散モデル**

## **7. 最適化のための各種関数及び最適化の関数を定義**

In [7]:
from scipy.optimize import minimize

# ポートフォリオの期待リターンを計算する関数
def portfolio_return(weights, expected_returns):
    return np.dot(weights, expected_returns)

# ポートフォリオの分散を計算する関数
def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

# 最適化する関数（分散を最小化）
def minimize_variance(weights, expected_returns, cov_matrix, target_return):
    ret = portfolio_return(weights, expected_returns)
    var = portfolio_variance(weights, cov_matrix)
    return var

# 制約条件
def constraint_return(weights, expected_returns, target_return):
    return portfolio_return(weights, expected_returns) - target_return

# ウェイトの合計が1になる制約
def constraint_sum(weights):
    return np.sum(weights) - 1

# 最適化の設定
def optimize_portfolio(expected_returns, cov_matrix, target_return):
    num_assets = len(expected_returns)
    args = (expected_returns, cov_matrix, target_return)
    constraints = (
        {'type': 'eq', 'fun': constraint_sum},
        {'type': 'eq', 'fun': constraint_return, 'args': (expected_returns, target_return)}
    )
    bounds = tuple((0, 1) for _ in range(num_assets))
    initial_weights = np.ones(num_assets) / num_assets

    result = minimize(minimize_variance, initial_weights, args=args, method='SLSQP', bounds=bounds, constraints=constraints)

    if result.success:
        return result.x
    else:
        raise ValueError("最適化が収束しませんでした。")



## **8. 特定の目標リターンのもと最適化を実行**

In [ ]:
# 目標リターンの設定（例: 0.002）
target_return = 0.002

# 最適ポートフォリオの計算
optimal_weights = optimize_portfolio(expected_returns, cov_matrix, target_return)

port_ret = portfolio_return(optimal_weights, expected_returns)
port_std = np.sqrt(portfolio_variance(optimal_weights, cov_matrix))

print("最適ポートフォリオのウェイト:")
for weight in optimal_weights:
    print(f"{weight:.2%}")

print(f"portfolio return = : {port_ret:.2%}")
print(f"portfolio standard deviation = : {port_std:.2%}")



## **9. 目標リターンを変更して最適ポートフォリオを推定、結果から効率的フロンティアのグラフを作成**

In [ ]:
target_return_list = [0.0015, 0.0017, 0.002, 0.0026, 0.003, 0.0035, 0.004, 0.0045, 0.00475, 0.005]

# リスクとリターンのリスト
portfolio_returns = []
portfolio_risks = []

# 各ターゲットリターンに対して最適化を実行
for target_return in target_return_list:
    optimal_weights = optimize_portfolio(expected_returns, cov_matrix, target_return)

    port_ret = portfolio_return(optimal_weights, expected_returns)
    port_std = np.sqrt(portfolio_variance(optimal_weights, cov_matrix))

    portfolio_returns.append(port_ret)
    portfolio_risks.append(port_std)

    print(f"portfolio return = {port_ret:.2%}")
    print(f"portfolio standard deviation = {port_std:.2%}")

# グラフのプロット
plt.figure(figsize=(10, 6))
plt.plot(portfolio_risks, portfolio_returns, 'o-')
plt.xlabel('Risk (Standard Deviation)')
plt.ylabel('Return')
plt.title('Efficient Frontier')
plt.grid(True)
plt.show()